In [60]:
import pandas as pd
import json
from pandas import json_normalize

In [61]:
# Load JSON
f = open('./mpd.slice.0-999.json')  
data = json.load(f)
f.close()

In [62]:
# Convert playlists array to df
df = pd.DataFrame(data['playlists'])
df.head()

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists
0,Throwbacks,false,0,1493424000,52,47,1,"[{'pos': 0, 'artist_name': 'Missy Elliott', 't...",6,11532414,37
1,Awesome Playlist,false,1,1506556800,39,23,1,"[{'pos': 0, 'artist_name': 'Survivor', 'track_...",5,11656470,21
2,korean,false,2,1505692800,64,51,1,"[{'pos': 0, 'artist_name': 'Hoody', 'track_uri...",18,14039958,31
3,mat,false,3,1501027200,126,107,1,"[{'pos': 0, 'artist_name': 'Camille Saint-Saën...",4,28926058,86
4,90s,false,4,1401667200,17,16,2,"[{'pos': 0, 'artist_name': 'The Smashing Pumpk...",7,4335282,16


In [63]:
# Clean unnecessary columns
df_cleaned = df.filter(['name','tracks'], axis=1)
df_cleaned.head()

,name,tracks
0,Throwbacks,"[{'pos': 0, 'artist_name': 'Missy Elliott', 't..."
1,Awesome Playlist,"[{'pos': 0, 'artist_name': 'Survivor', 'track_..."
2,korean,"[{'pos': 0, 'artist_name': 'Hoody', 'track_uri..."
3,mat,"[{'pos': 0, 'artist_name': 'Camille Saint-Saën..."
4,90s,"[{'pos': 0, 'artist_name': 'The Smashing Pumpk..."


In [84]:
# Create final df with nested track's array in each playlist
df_final = pd.DataFrame()
for i in range(0, len(data['playlists'])):
    df_nested_list = pd.json_normalize(
        data['playlists'][i], 
        record_path =['tracks'],
        meta=['name']
    )
    df_cleaned2 = df_nested_list.filter(['name','album_name','track_name','artist_name'], axis=1)
    df_final = pd.concat([df_final, df_cleaned2], ignore_index=True)
df_final.rename(columns = {'name':'list_name'}, inplace = True)
df_final = df_final.filter(['list_name','track_name'], axis=1)
df_final.head(60)

,list_name,track_name
0,Throwbacks,Lose Control (feat. Ciara & Fat Man Scoop)
1,Throwbacks,Toxic
2,Throwbacks,Crazy In Love
3,Throwbacks,Rock Your Body
4,Throwbacks,It Wasn't Me
5,Throwbacks,Yeah!
6,Throwbacks,My Boo
7,Throwbacks,Buttons
8,Throwbacks,Say My Name
9,Throwbacks,Hey Ya! - Radio Mix / Club Mix


In [46]:
# Convert df in oneHotEncoded for tracks
from sklearn.preprocessing import LabelBinarizer

jobs_encoder = LabelBinarizer()
jobs_encoder.fit(df_final['track_name'])
transformed = jobs_encoder.transform(df_final['track_name'])
ohe_df = pd.DataFrame(transformed)
data_ohe = pd.concat([df_final, ohe_df], axis=1).drop(['track_name'], axis=1)
data_ohe.head(60)

,list_name,0,1,2,3,4,5,6,7,8,...,1373,1374,1375,1376,1377,1378,1379,1380,1381,1382
0,Throwbacks,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Throwbacks,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Throwbacks,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Throwbacks,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Throwbacks,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Throwbacks,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Throwbacks,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Throwbacks,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Throwbacks,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Throwbacks,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
# Convert df in oneHotEncoded for tracks with track_names as columns (Cleaning name)
from sklearn.preprocessing import OneHotEncoder

encoder=OneHotEncoder(sparse=False)
train_X_encoded = pd.DataFrame(encoder.fit_transform(df_final[['track_name']]))
train_X_encoded.columns = encoder.get_feature_names_out(['track_name'])
for index in range(0, len(train_X_encoded.columns)):
    string = str(train_X_encoded.columns[index])
    replace = string.replace('track_name_','')
    train_X_encoded.columns.values[index] = replace
df_final.drop(['track_name'] ,axis=1, inplace=True)
OH_X_train= pd.concat([df_final, train_X_encoded ], axis=1)
OH_X_train.head(60)

,list_name,$ave Dat Money (feat. Fetty Wap & Rich Homie Quan),(Everything I Do) I Do It For You,(Not So) Sad And Lonely,1 Billion Dogs,1-800-273-8255,10:1,13,19,1904,...,pick up the phone,suncream,way it goes,wokeuplikethis*,wonderful,꺼내 먹어요 (Eat),두근거려 (Beautiful),사실은 The Truth Is,양화대교 (Yanghwa Brdg),전야 前夜 The Eve
0,Throwbacks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Throwbacks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Throwbacks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Throwbacks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Throwbacks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Throwbacks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Throwbacks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Throwbacks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Throwbacks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Throwbacks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
